In [17]:
import pybedtools.bedtool as pybed
import pybedtools
import os
import pandas as pd
from tqdm.notebook import tqdm
import math
from statistics import mode

#### Input and output dir

In [18]:
workdir="/scratch/rx32940/minION/polyA_directRNA/TU_Annotation"

input=os.path.join(workdir, "input")
output=os.path.join(workdir, "example")

#### Bam to bed
- read in gene annotation file, which contains information about chromsome ("chrom"),gene start site ("start"), end site ("end"), strand ("strand"), gene locus tag ("locus tag"), gene name ("name") 
- read in bam/bed file (if bed file not exist, read bam and convert to bed)

In [19]:
gene_file=os.path.join(input, "GCF_000007685.1_gene_table.txt") # gene annotation file

bam="/scratch/rx32940/minION/polyA_cDNA/map/genome/bam/Copenhageni_Basecalled_Aug_16_2019_Direct-cDNA_PolyATail_rna_filtered.bam"

 # bam
bed_file=os.path.join(input, "Copenhageni_Basecalled_Aug_16_2019_Direct-cDNA_PolyATail_rna_filtered.bed") # bed

if not os.path.isfile(bed_file): # if bed file not exist? convert from bam
    bam = pybed.BedTool(bam)
    bed=bam.bam_to_bed().to_dataframe()
    bed.to_csv(bed_file,sep="\t", index = False)
else: # read bed file
    bed=pd.read_csv(bed_file, sep="\t")



#### process input files

- only keep reads with mapping quality >=30
- separate reads mapped to leading and lagging strand
- read in gene annotations
- separate genes on the leading and lagging strand

In [20]:
bed_filtered = bed#.loc[bed["score"] >= 30] # filter read quality

read_map_lead=bed_filtered.loc[bed["strand"] == "+"]
read_map_lag = bed_filtered.loc[bed["strand"] == "-"]

gene_anot=pd.read_csv(gene_file, sep="\t")

lead_strand_genes = gene_anot.loc[gene_anot["strand"] == "+"]
lag_strand_genes = gene_anot.loc[gene_anot["strand"] == "-"]

### Gene ORF aware TSS identification
- input: 
    1) gene start codon annotation
    2) bed file converted from bam alignment file
- output: 
    1) potential TSS sites for each gene, number of reads at the each TSS site range, maximum end site stretch for all reads mapped at the site


#### lead TSS with gene start codon annotation

In [21]:
threshold = 10 # the position of reads with start codon less than threshold are put into the same cluster 

chroms = gene_anot["genomic_accession"].unique() # chromosomes in the genome


In [22]:
# function to get reads overlap at either "start" or "end" (start in lagging strand) of a gene
def get_overlap_reads(read_mapped_cur_chrom, cur_chrom_gene ,gene, position):
    
    gene_start = int(cur_chrom_gene.loc[cur_chrom_gene["locus_tag"]==gene,position]) # start codon of the current gene at 5' end
    reads_before_start = read_mapped_cur_chrom.loc[read_mapped_cur_chrom["start"] <= gene_start] # reads with start sites mapped before gene start codon
    reads_overlap_start = reads_before_start.loc[reads_before_start["end"] >= gene_start] # from reads start before start codon, get reads overlapped with the start codon (ends after start codon)
    reads_overlap_start.reset_index(drop=True, inplace=True)
    return reads_overlap_start



In [35]:
def get_tss_pos(reads_overlap, cluster_start, cluster_end,position):
    start_list = reads_overlap.loc[cluster_start:cluster_end, "start"].tolist()
    tss_start= mode(start_list) if start_list.count(mode(start_list)) > 2 else min(start_list)
    end_list=reads_overlap.loc[cluster_start:cluster_end, "end"].tolist()
    tss_end = mode(end_list) if end_list.count(mode(end_list)) > 2 else max(end_list)# find the end sites (read with furtherest mapping) of the current cluster
    if position == "start":
        return tss_start+1, tss_end
    else:
        return tss_end, tss_start+1
            
def cluster_tss(reads_overlap,position, gene_name, chrom):
    cur_gene_tss_clusters = pd.DataFrame() # tss clusters at for current gene's start codon
    
    
    reads_overlap = reads_overlap.sort_values(by=position) # sort all reads overlapped with current gene's start codon based on their start sites
    reads_overlap.reset_index(drop=True, inplace=True)
    

    cluster_start = 0 # start position of the cluster  
    cluster_end = 0 # start position of the cluster  (not included)
    for index in range(1,len(reads_overlap.iloc[:,1])):
        current_read = reads_overlap.iloc[index]
        if (current_read[position] - reads_overlap.loc[index-1,position]) <= threshold: # if the start sites of two reads are close enough 
            cluster_end += 1 # assign to the same cluster
        else:# if the start site of reads are too distant from each other to be in the same cluster
            tss_pos = get_tss_pos(reads_overlap,cluster_start, cluster_end , position)
            cur_gene_tss_clusters = cur_gene_tss_clusters.append({"chrom" : cur_chrom,"start" : tss_pos[0], "end":tss_pos[1], "gene" : cur_gene,"num_reads":int(cluster_end - cluster_start)},ignore_index=True)
            cluster_start =  cluster_end = index 
            index += 1


    # the last tss cluster when rest of the reads didn't get to go into the else condition to write their cluster into the df
    tss_pos = get_tss_pos(reads_overlap, cluster_start, cluster_end, position)
    cur_gene_tss_clusters = cur_gene_tss_clusters.append({"chrom" : chrom,"start" : tss_pos[0], "end":tss_pos[1], "gene" : gene_name,"num_reads":int(cluster_end - cluster_start)},ignore_index=True)

    return cur_gene_tss_clusters

In [36]:
lead_gene_tss_clusters = pd.DataFrame()

for cur_chrom in chroms:
    
    cur_chrom_gene = lead_strand_genes.loc[lead_strand_genes["genomic_accession"] == cur_chrom]
    cur_chrom_gene.reset_index(drop=True, inplace=True)
    
    read_mapped_cur_chrom = read_map_lead.loc[read_map_lead["chrom"]== cur_chrom]
    
    for gene in tqdm(range(len(cur_chrom_gene.iloc[:,1]))): # iterate through all genes
        cur_gene = cur_chrom_gene.loc[gene,"locus_tag"] # current gene name
        reads_overlap_start = get_overlap_reads(read_mapped_cur_chrom, cur_chrom_gene ,cur_gene, "start")
        cov_at_pos = len(reads_overlap_start.iloc[:,1]) # find the coverage of the current gene's start codon, (all the reads overlap at gene start codon)
        
        if cov_at_pos > 1: # if the gene was mapped
            cur_gene_tss_clusters = cluster_tss(reads_overlap_start, "start", cur_gene, cur_chrom)
            lead_gene_tss_clusters  = lead_gene_tss_clusters.append(cur_gene_tss_clusters[(cur_gene_tss_clusters["num_reads"] >= (cov_at_pos * 0.3)) | (cur_gene_tss_clusters["num_reads"] >= 3)])
            


KeyboardInterrupt: 

In [ ]:
# block for testing
# cur_chrom="NC_005823.1"
# cur_chrom_gene = lead_strand_genes.loc[lead_strand_genes["genomic_accession"] == cur_chrom]
# cur_chrom_gene.reset_index(drop=True, inplace=True)

# read_mapped_cur_chrom = read_map_lead.loc[read_map_lead["chrom"]== cur_chrom]
# gene=298
# cur_gene = cur_chrom_gene.loc[gene,"locus_tag"]
# reads_overlap_start = get_overlap_reads(read_mapped_cur_chrom, cur_chrom_gene ,cur_gene, "start")
# cov_at_pos = len(reads_overlap_start.iloc[:,1])
# if cov_at_pos > 1: # if the gene was mapped
#     cur_gene_tss_clusters = cluster_tss(reads_overlap_start, "start",cur_gene ,cur_chrom)

# cur_gene_tss_clusters[cur_gene_tss_clusters["num_reads"] > cov_at_pos * 0.3]


In [ ]:
# write lead strand tss annotation with start codon awareness into file
lead_gene_tss_clusters["strand"] = "+" # write lead strand gene's tss read clusters 


#### Gene aware TSS annotation on lagging strand

In [ ]:
lag_gene_tss_clusters = pd.DataFrame()

for cur_chrom in chroms:
    
    cur_chrom_gene = lag_strand_genes.loc[lag_strand_genes["genomic_accession"] == cur_chrom]
    cur_chrom_gene.reset_index(drop=True, inplace=True)
    
    read_mapped_cur_chrom = read_map_lag.loc[read_map_lag["chrom"]== cur_chrom]
    
    for gene in tqdm(range(len(cur_chrom_gene.iloc[:,1]))): # iterate through all genes
        cur_gene = cur_chrom_gene.loc[gene,"locus_tag"] # current gene name
        reads_overlap_end = get_overlap_reads(read_mapped_cur_chrom, cur_chrom_gene ,cur_gene, "end")
        cov_at_pos = len(reads_overlap_end.iloc[:,1]) # find the coverage of the current gene's start codon, (all the reads overlap at gene start codon)
        
        if cov_at_pos > 1: # if the gene was mapped
            cur_gene_tss_clusters = cluster_tss(reads_overlap_end, "end", cur_gene, cur_chrom)

        lag_gene_tss_clusters  = lag_gene_tss_clusters.append(cur_gene_tss_clusters[cur_gene_tss_clusters["num_reads"] > cov_at_pos * 0.3])
            


In [ ]:
lag_gene_tss_clusters["strand"]="-"


In [ ]:
gene_aware_tss = pd.concat([lead_gene_tss_clusters, lag_gene_tss_clusters], ignore_index = True)

gene_aware_tss = gene_aware_tss.dropna() # remove nan in the dataframe
gene_aware_tss= gene_aware_tss.loc[gene_aware_tss["num_reads"] > 2]

gene_aware_tss= gene_aware_tss.astype({'start':int, 'end':int, "num_reads":int})
gene_aware_tss.to_csv(os.path.join(output,"gene_aware_tss.tab"), sep="\t", index=False)

### TSS without gene start codon annotations

In [ ]:
#Identification of TSS with no annotation

read_map_lead = bed_filtered.loc[bed_filtered["strand"] == "+"]
read_map_lag = bed_filtered.loc[bed_filtered["strand"] == "-"]


In [ ]:
            
def cluster_tss_unaware(cur_chrom_reads,position):
    cur_tss_clusters = pd.DataFrame()
    cluster_start = 0 # start position of the cluster  
    cluster_end = 0 # start position of the cluster  (not included)
    for index in tqdm(range(1,len(cur_chrom_reads.iloc[:,1]))): # loop through reads
        current_read = cur_chrom_reads.iloc[index]
        if (current_read[position] - cur_chrom_reads.loc[index-1,position]) <= threshold: # if the start sites of two reads are close enough 
            cluster_end += 1 # assign to the same cluster
        else:# if the start site of reads are too distant from each other to be in the same cluster
            tss_pos = get_tss_pos(cur_chrom_reads,cluster_start, cluster_end , position)
            cur_tss_clusters = cur_tss_clusters.append({"chrom" : cur_chrom,"start" : tss_pos[0], "end":tss_pos[1], "name":"gene_unaware","num_reads":int(cluster_end - cluster_start  + 1)},ignore_index=True)
            cluster_start = cluster_end = index 
            index += 1

    # the last tss cluster when rest of the reads didn't get to go into the else condition to write their cluster into the df
    tss_pos = get_tss_pos(cur_chrom_reads, cluster_start, cluster_end, position)
    cur_tss_clusters = cur_tss_clusters.append({"chrom" : cur_chrom,"start" : tss_pos[0], "end":tss_pos[1], "name":"gene_unaware","num_reads":int(cluster_end - cluster_start  + 1)},ignore_index=True)

    return cur_tss_clusters

In [ ]:
tss_clusters = pd.DataFrame() # tss clusters at for all reads' start codon


for cur_chrom in chroms:
    
    # read mapped to one chromosome
    read_map_cur_chrom_lead = read_map_lead.loc[read_map_lead["chrom"] == cur_chrom]

    read_map_cur_chrom_lead = read_map_cur_chrom_lead.sort_values(by="start") # sort all reads overlapped with current gene's start codon based on their start sites
    read_map_cur_chrom_lead.reset_index(drop=True, inplace = True)
    cur_tss_clusters = cluster_tss_unaware(read_map_cur_chrom_lead, "start")

    tss_clusters =tss_clusters.append(cur_tss_clusters ,ignore_index=True)
    


In [ ]:
# # this block is used for testing
# read_map_cur_chrom_lead = read_map_lead.loc[read_map_lead["chrom"] == cur_chrom]
# read_map_cur_chrom_lead = read_map_cur_chrom_lead.sort_values(by="start")
# read_map_cur_chrom_lead.reset_index(drop=True, inplace = True)
# index=91925
# current_read = read_map_cur_chrom_lead.iloc[index]
# cur_tss_clusters = pd.DataFrame()
# cluster_start = 0 # start position of the cluster  
# cluster_end = 0 # start position of the cluster  (not included)
# for index in tqdm(range(index,91980)): # loop through reads
#     current_read = read_map_cur_chrom_lead.iloc[index]
#     if (current_read["start"] - read_map_cur_chrom_lead.loc[index-1,"start"]) <= threshold: # if the start sites of two reads are close enough 
#         cluster_end += 1 # assign to the same cluster
#     else:# if the start site of reads are too distant from each other to be in the same cluster
#         tss_pos = get_tss_pos(read_map_cur_chrom_lead,cluster_start, cluster_end , "start")
#         cur_tss_clusters = cur_tss_clusters.append({"chrom" : cur_chrom,"start" : tss_pos[0], "end":tss_pos[1], "name":"gene_unaware","num_reads":int(cluster_end - cluster_start  + 1)},ignore_index=True)
#         cluster_start = cluster_end = index 
#         index += 1

In [ ]:
lead_tss_clusters = tss_clusters
lead_tss_clusters.reset_index(drop=True, inplace = True)
lead_tss_clusters["strand"] = "+"
lead_tss_clusters= lead_tss_clusters.astype({'start':int, 'end':int, "num_reads":int})
lead_tss_clusters.to_csv(os.path.join(output, "lead.strand.tss.gene.unaware.bed"), sep="\t", index = False)

In [ ]:
tss_clusters = pd.DataFrame() # tss clusters at for all reads' start codon (lagging strand at 3' end)


for cur_chrom in chroms:
    
    # read mapped to one chromosome
    read_map_cur_chrom_lag = read_map_lag.loc[read_map_lag["chrom"] == cur_chrom]

    read_map_cur_chrom_lag = read_map_cur_chrom_lag.sort_values(by="end") # sort all reads overlapped with current gene's start codon based on their start sites
    read_map_cur_chrom_lag.reset_index(drop=True, inplace = True)
    cur_tss_clusters = cluster_tss_unaware(read_map_cur_chrom_lag, "end")

    tss_clusters =tss_clusters.append(cur_tss_clusters ,ignore_index=True)


In [ ]:
lag_tss_clusters = tss_clusters
lag_tss_clusters.reset_index(drop=True, inplace = True)
lag_tss_clusters["strand"] = "-"
lag_tss_clusters= lag_tss_clusters.astype({'start':int, 'end':int, "num_reads":int})
lag_tss_clusters.to_csv(os.path.join(output, "lag.strand.tss.gene.unaware.bed"), sep="\t", index = False)

In [ ]:
gene_unaware_tss= pd.concat([lead_tss_clusters,lag_tss_clusters], ignore_index = True)
gene_unaware_tss.to_csv(os.path.join(output, "tss.gene.unaware.tab"), sep="\t", index = False)

#### Filter gene unaware TSS based on cov

In [ ]:
# separate reads mapped to leading and lagging strand in the input bam file
lead_bed = bed_filtered.loc[bed["strand"] == "+"]
lag_bed = bed_filtered.loc[bed["strand"] == "-"]

# write reads mapped to each bam file in bed (separate file for each stand)
lead_bed.to_csv(os.path.join(output, "bamtobed_lead.bed"), header=False,sep="\t", index=False)
lag_bed.to_csv(os.path.join(output, "bamtobed_lag.bed"), header=False,sep="\t", index = False)

# use bed tools to read in each strand's bed file
lead_bam_bed = pybed.BedTool(os.path.join(output, "bamtobed_lead.bed"))
lag_bam_bed = pybed.BedTool(os.path.join(output, "bamtobed_lag.bed"))




In [ ]:
# calculate coverage at each base for each strand (use bedtools genome_coverage)
lead_bam_cov = lead_bam_bed.genome_coverage(g=os.path.join(input, "GCA_000007685.1_genome_file.txt"), d=True)
lag_bam_cov = lag_bam_bed.genome_coverage(d=True,g=os.path.join(input, "GCA_000007685.1_genome_file.txt"))

lead_bam_cov_df = pd.read_table(lead_bam_cov.fn, names =[ "chrom", "start", "cov"] )
lag_bam_cov_df = pd.read_table(lag_bam_cov.fn, names=["chrom", "start", "cov"])


In [ ]:
# write per-base cov to bed file
lead_bam_cov_df.to_csv(os.path.join(output, "bamtobed_lead_cov.bed"), header=False,sep="\t", index=False)
lag_bam_cov_df.to_csv(os.path.join(output, "bamtobed_lag_cov.bed"), header=False,sep="\t", index=False)


In [ ]:
# this is unaware tss sites get from previous 
# lead_tss_clusters = pd.read_csv(os.path.join(output, "lead.strand.tss.gene.unaware.bed"),sep="\t",index_col=False) # can delete later
# lag_tss_clusters = pd.read_csv(os.path.join(output, "lag.strand.tss.gene.unaware.bed"),sep="\t",index_col=False)


In [ ]:
# get cov of the site of each detected tss
lead_unaware_tss_start_cov = lead_tss_clusters.merge(lead_bam_cov_df,how="left", on=["chrom", "start"]) 
lead_unaware_tss_filtered= lead_unaware_tss_start_cov.loc[lead_unaware_tss_start_cov["num_reads"] >= lead_unaware_tss_start_cov["cov"]]


lag_unaware_tss_start_cov = lag_tss_clusters.merge(lag_bam_cov_df,how="left", on=["chrom", "start"]) 
lag_unaware_tss_filtered= lag_unaware_tss_start_cov.loc[lag_unaware_tss_start_cov["num_reads"] >= lag_unaware_tss_start_cov["cov"]]


In [ ]:
# combine lead and lag strand, write filtered tss to file
lead_unaware_tss_filtered.reset_index(drop=True, inplace=True)
lag_unaware_tss_filtered.reset_index(drop=True, inplace=True)
gene_unaware_tss_filtered= pd.concat([lead_unaware_tss_filtered, lag_unaware_tss_filtered])
gene_unaware_tss_filtered.to_csv(os.path.join(output,"gene_unaware_tss_filtered.tab"), sep = "\t", index=False)

### Combine gene aware and unaware TSS

In [ ]:
# can delete
# gene_aware_tss = pd.read_csv(os.path.join(output, "gene_aware_tss.tab"), index_col=False, sep="\t")

In [ ]:
combine_approaches = gene_unaware_tss_filtered[["chrom","start", "end", "strand"]].merge(gene_aware_tss, how="outer", on=["chrom","start","strand"])
combine_approaches["end"] = combine_approaches.apply(lambda row: row["end_y"] if math.isnan(row["end_x"]) else row["end_x"],axis=1)
combine_approaches_edited = combine_approaches.drop(["end_x", "end_y","num_reads"],axis=1)
combine_approaches_edited = combine_approaches_edited.astype({"start":int, "end":int})
combine_approaches_edited.to_csv(os.path.join(output, "combined_tss.tab"), index=False, sep="\t")

In [ ]:
combine_approaches.apply(lambda row: math.is_nan(row["end_y"]),axis=1)

In [ ]:
input="/scratch/rx32940/minION/polyA_directRNA/TU_Annotation/input"
output="/scratch/rx32940/minION/polyA_directRNA/TU_Annotation/example"
# gene file with the annotated genes, a table where chromosome is in the first column, start codon position in the fourth column, stop codon in the fifth, sign in the seventh, name in the ninth
gene_file = os.path.join(input, "GCF_000007685.1_gene_table_test.txt")
#input file is a bed file generated from sorted bam alignment file using bedtools bamtobed
input_file = os.path.join(input, "Copenhageni_Basecalled_Aug_16_2019_Direct-cDNA_NoPolyATail_rna_filtered_test.bed")
output_file_aware = os.path.join(output, "test.gene_aware.tab")
#threshold is in general dependent on the sequencing depth
threshold = 10

In [ ]:
with open(gene_file, 'r') as gen, open(input_file, 'r') as bed, open(output_file_aware, 'w') as new:
    all_reads_neg = {'NC_005823.1': [], 'NC_005824.1': []}  # both genes and reads in their chromosome
    all_reads_pos = {'NC_005823.1': [], 'NC_005824.1': []}
    genes = {'NC_005823.1': [], 'NC_005824.1': []} #only genes
    for line in gen: #store genes coordinates first
        read = line.strip().split("\t")
        if read[6] == '-':
            all_reads_neg[read[0]].append((int(read[3]), int(read[4]), read[8], 0)) #zero in the end indicates that it is a gene not a read
        else:
            all_reads_pos[read[0]].append((int(read[3]), int(read[4]), read[8], 0))
    

all_reads_pos['NC_005823.1'][0:10]

        

In [ ]:
with open(gene_file, 'r') as gen, open(input_file, 'r') as bed, open(output_file_aware, 'w') as new:
    for line in bed: #store reads coordinates
        read = line.strip().split()
        if read[5] == '-':
            all_reads_neg[read[0]].append((int(read[1]), int(read[2]), 'read', 1, 1)) #1 to indicate it is a read not a gene
        else:
            all_reads_pos[read[0]].append((int(read[1]), int(read[2]), 'read', 1, 1))
# all_reads_pos['NC_005823.1'][len(all_reads_pos['NC_005823.1'])-5: len(all_reads_pos['NC_005823.1'])]

In [ ]:
all_reads_pos['NC_005823.1'][len(all_reads_pos['NC_005823.1'])-5: len(all_reads_pos['NC_005823.1'])]

In [ ]:
with open(gene_file, 'r') as gen, open(input_file, 'r') as bed, open(output_file_aware, 'w') as new:
    for ch in all_reads_pos: #process positive reads and genes seperately from negative ones
            chromosome = all_reads_pos[ch]
            chromosome.sort() #sort genes and reads according to their start site
            read_index = 0
            prev_gene = 0
            for read in chromosome: #read here can be either a gene or a read
                if read[-1] == 0: # stop if it is a gene
                    before = list(chromosome[0:read_index]) #take the reads before the start codon
                    before.sort(key=lambda before: before[1]) #sort them accoding to their end site
                    end = before[-1][1] #the furthest end
                    if end > read[0]: # if it is after the start codon i.e. read overlaps the start codon
                        after_index = len(before) - 1
                        # take reads until their end bcome less than the start codon (don't overlap the start codon anymore = )
                        while end > read[0] and after_index > 0:
                            after_index -= 1 #move to the next read
                            end = before[after_index][1] #set the end of the next read, it will be checked in the next round of the while function
                        after = list(before[after_index:])# Only the reads that overlap the start codon
                        #now resolve if their multiple peaks
                        current = list()
                        ends = list()
                        cov = len(after) # the coverage at the start codon is the number of overlapping reads
                        after.sort() #sort again according to the start site
                        prev = after[0][0]
                        for end in after: #end is actually the read not just an end
                            if end[0] - prev < threshold: #if the distance between the read start site and the next one is less than the threshold, take in the same cluster
                                current.append(end[0])
                                ends.append(end[1])
                                prev = end[0]
                            else:
                                if len(current) > (0.3 * cov):  # if the read is far from than the next one, finish the current cluster and check it if its count is important
                                    ends.sort() # optional step to calculate the end of these start sites
                                    tss_end = ends[-1] # the end is the most extrem end
                                    new.write('%s\t%i\t%i\t%s\t+\t%i\n' % (ch, current[0] + 1, len(current), read[2], tss_end))
                                current = [end[0]] #start a new cluster with the current read
                                ends = [end[1]]
                                prev = end[0]
                    if len(current) > (0.3 * cov) and len(current) > 2: #to check the last cluster
                        # print(read[2])
                        ends.sort()
                        tss_end = ends[-1]
                        new.write('%s\t%i\t%i\t%s\t+\t%i\n' % (ch, current[0] + 1, len(current), read[2], tss_end))
                read_index += 1

In [ ]:
with open(gene_file, 'r') as gen, open(input_file, 'r') as bed, open(output_file_aware, 'a') as new:
    for ch in all_reads_neg: #similar to above but designed for negative reads
            chromosome = all_reads_neg[ch]
            chromosome.sort(key=lambda chromosome: chromosome[1])
            read_index = 0
            for read in chromosome:
                if read[-1] == 0:
                    # after_idx = read_index
                    current = list()
                    for i in chromosome[read_index + 1:]:
                        if i[0] > read[1]:
                            break
                        elif i[1] > read[1]:
                            current.append((i[1], i[0]))  # i[1]
                    if len(current) > 0:

                        current.sort()
                        prev = current[0][0]
                        cov = len(current)
                        curr = list()
                        ends = list()
                        for i in current:
                            if (i[0] - prev) < 10:
                                curr.append(i[0])
                                prev = i[0]
                                ends.append(i[1])
                            else:
                                if len(curr) > (0.3 * cov):
                                    ends.sort()
                                    tss_end = ends[0]
                                    new.write('%s\t%i\t%i\t%s\t-\t%i\n' % (ch, curr[-1] + 1, len(curr), read[2], tss_end))
                                curr = [i[0]]
                                prev = i[0]
                        if len(curr) > (0.3 * cov) and len(curr) > 2:
                            ends.sort()
                            tss_end = ends[0]
                            new.write('%s\t%i\t%i\t%s\t-\t%i\n' % (ch, curr[-1] + 1, len(curr), read[2], tss_end))
                read_index += 1

In [ ]:
#Identification of TSS with no annotation
intermediate_file = os.path.join(output, "TSS_id_no_annotation")
#input file is a bed file generated from sorted bam alignment file using bedtools bamtobed
input_file = input_file
with open(input_file, 'r') as old, open(intermediate_file, 'w') as new:
    pos_reads = {'NC_005823.1': [], 'NC_005824.1': []} #seperate positive from negative reads
    neg_reads = {'NC_005823.1': [], 'NC_005824.1': []}
    for line in old:
        read = line.strip().split()
        if read[-1] == '+':
            pos_reads[read[0]].append(int(read[1]))
        else:
            neg_reads[read[0]].append(int(read[2]))
    for chromosome in pos_reads:
        chromo = pos_reads[chromosome]
        chromo.sort()
        prev = chromo[0]
        current = [chromo[0]]
        for site in chromo[1:]:
            if site - prev < threshold:
                current.append(site)
                prev = site
            else:
                if len(current) > 3: #if the cluster is more than three reads considere it for further analysis
                    new.write('%s\t%i\t%i\t%i\t+\n' % (chromosome, current[0]+1, len(current), current[-1] - current[0]))
                prev = site
                current = [site]

In [ ]:
output_file_unaware = os.path.join(output,'filtered_TSS_unaware.tab')
pos_cov = os.path.join(output,'bamtobed_lead_cov.bed')
neg_cov = os.path.join(output,'bamtobed_lag_cov.bed')

with open(intermediate_file, 'r') as old, open(output_file_unaware, 'w') as new, open(pos_cov, 'r') as pos, open(neg_cov, 'r') as neg:
    coverage = {'-': {'NC_005823.1': [], 'NC_005824.1': []}, '+': {'NC_005823.1': [], 'NC_005824.1': []}} #store coverage
    for line in pos:
        read = line.strip().split()
        coverage['+'][read[0]].append(int(read[2]))
    for line in neg:
        read = line.strip().split()
        coverage['-'][read[0]].append(int(read[2]))
    for line in old:
        read = line.strip().split()
        if int(read[2]) > coverage[read[-1]][read[0]][int(read[1])]:  # if cluster meet the coverage criteria leave it, else don't print it
            new.write(line)

In [ ]:
coverage[read[-1]][read[0]][int(983)]